In [1]:
#claissifcation sorting
import numpy as np
import scipy
import xgboost as xgb
import pickle as pk
import math
from sklearn.linear_model import LinearRegression

In [54]:
#get the data from splitted validate / train set
train_data = np.loadtxt("./train.txt")
valid_data = np.loadtxt("./valid.txt")

train_y = train_data[:, 0].astype(int)
train_x = train_data[:, 1:94].astype(int)
valid_y = valid_data[:, 0].astype(int)
valid_x = valid_data[:, 1:94].astype(int)
print(valid_y)
xgb_train = xgb.DMatrix(train_x, label= train_y)
xgb_valid = xgb.DMatrix(valid_x, label= valid_y)

[2 6 3 ... 8 9 6]


In [55]:
#set parameter for xgboost
param = {}
param['objective'] = 'multi:softmax'
param['eta'] = 0.1
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 10

In [57]:
watchlist = [(xgb_train, 'train')]
num_round = 10
boost = xgb.train(param, xgb_train, num_round, watchlist)

[0]	train-merror:0.271332
[1]	train-merror:0.255246
[2]	train-merror:0.247822
[3]	train-merror:0.240423
[4]	train-merror:0.235575
[5]	train-merror:0.233277
[6]	train-merror:0.230954
[7]	train-merror:0.227999
[8]	train-merror:0.226206
[9]	train-merror:0.225424


In [66]:
#get prediction for accuracy
pred = boost.predict(xgb_valid)
accuracy = np.sum(pred == valid_y) / valid_y.shape[0]
print('Validation accuracy using softmax = {}' .format(accuracy))

Validation error using softmax = 0.7591152408847591


In [67]:
#get result probabilities
param['objective'] = 'multi:softprob'
bst = xgb.train(param, xgb_train, num_round, watchlist)

pred_prob = bst.predict(xgb_valid).reshape(valid_y.shape[0], 10)
pred_label = np.argmax(pred_prob, axis = 1)
accuracy = np.sum(pred_label != valid_y) / valid_y.shape[0]
print('Validation accuracy using softprob = {} '.format(accuracy))

[0]	train-merror:0.271332
[1]	train-merror:0.255246
[2]	train-merror:0.247822
[3]	train-merror:0.240423
[4]	train-merror:0.235575
[5]	train-merror:0.233277
[6]	train-merror:0.230954
[7]	train-merror:0.227999
[8]	train-merror:0.226206
[9]	train-merror:0.225424
Validation accuracy using softprob = 0.24088475911524088 


In [74]:
print(pred_prob.shape)

(9901, 10)


In [85]:
#classification result sorting
result = []
for i in range(0, pred_prob.shape[0]):
    prob_label = []
    for j in range(0, pred_prob.shape[1]):
        prob_label.append(str(pred_prob[i][j]) + ":" + str(j + 1))
    
    prob_label.sort()
    result.append(prob_label) 
print(result[0])

['0.0572055:1', '0.05747394:6', '0.060451668:5', '0.06096526:2', '0.061381795:8', '0.0631573:7', '0.06463116:9', '0.071801804:10', '0.13775776:4', '0.36517385:3']


In [93]:
#count class_number
validYCount = [0 for i in range(0, 9)]
for v in valid_y:
    validYCount[v - 1] += 1

print(validYCount)
total = np.sum(validYCount)
#

[306, 2568, 1259, 426, 426, 2226, 449, 1414, 827]
9901
